# Neural Network

## 딥러닝 맛보기

- 추후 사용할 RNN을 위하여 딥러닝 즉 신경망에 대해 맛보기하는 시간을 가저보자
- 이전 다른 분류기도 그랬지만 분류와 회귀 모두에서 사용할 수 있다.
- 수학적 원리는 정확히 설명하고 구현하기 어려우므로 다른 자료를 참조하기를 권장함
- 기본 구조 외에 다양한 architecture에 대한 것은 [여기](https://github.com/young31/Deep-Learning/tree/master/Architecture)
  
    
- keras를 사용하여 구현할 예정
    - keras는 쉽게 사용하게 해주는 툴
    - tensorflow 등을 백으로 가질 수 있음
    
- 순차모형과 이름을 달아 연결하는 방법이 있음
    - 순차모형이 시작하기는 쉬움
    - 나중에 복잡한 망을 짜려면 개별 망에 연결하는 방법을 사용
    
- 신경망은 greedy하게 해를 찾는 경사하강법을 사용해서 답을 찾아가므로 정규화를 무조건 해줘야함!!!
    - 해당 절에서는 편의를 위해 카테고리변수를 따로 설정하지 않음

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv('../1.clustering/titanic.csv')

target = data['Survived']
data = data.drop(['PassengerId', 'Survived'], axis=1)

def sex(a):
    if a == 'male':
        return 0
    else:
        return 1
    
def emb(a):
    if a == 'S':
        return 0
    elif a == 'Q':
        return 1
    else:
        return 2
    
data['Sex'] = data['Sex'].map(sex)
data['Embarked'] = data['Embarked'].map(emb)

train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=2019, shuffle=True)

In [2]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
tt = std.fit_transform(train_X)
te = std.fit_transform(test_X)

In [3]:
## 층을 만든다
from keras import models, layers, optimizers
import keras.backend as K
K.clear_session()

input_layer = layers.Input(shape=train_X.shape[1:])

layer1 = layers.Dense(32, activation='relu')(input_layer)
layer2 = layers.Dense(16, activation='relu')(layer1)

out_layer = layers.Dense(1, activation='sigmoid')(layer2)

model = models.Model(input_layer, out_layer)

Using TensorFlow backend.


In [4]:
## compile 한다.

model.compile(optimizer = optimizers.Adam(lr=0.01),
             loss='binary_crossentropy',
             metrics=['acc'])

In [5]:
# ## custom loss
# from keras import backend as K
# import tensorflow as tf

# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
#         return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed
# ## cunstom metric

# def f1(y_true, y_pred):
#     def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))
# custom_loss = focal_loss()

In [6]:
# model.compile(optimizer = optimizers.Adam(lr=0.1),
#              loss= custom_loss,
#              metrics=['acc'])

In [7]:
## callback 함수를 쓴다면 추가
from keras.callbacks import EarlyStopping, ModelCheckpoint

cp = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

In [8]:
## 훈련시킨다.
model.fit(tt, train_y,
         epochs=100,
         validation_split=0.3,
         callbacks=[cp, es])


Train on 348 samples, validate on 150 samples
Epoch 1/100
348/348 [==============================] - 0s 1ms/step - loss: 0.5331 - acc: 0.7586 - val_loss: 0.4402 - val_acc: 0.8200
Epoch 2/100
348/348 [==============================] - 0s 97us/step - loss: 0.4417 - acc: 0.8103 - val_loss: 0.4445 - val_acc: 0.8133
Epoch 3/100
348/348 [==============================] - 0s 97us/step - loss: 0.4240 - acc: 0.8190 - val_loss: 0.4505 - val_acc: 0.8133
Epoch 4/100
348/348 [==============================] - 0s 97us/step - loss: 0.4067 - acc: 0.8391 - val_loss: 0.4309 - val_acc: 0.8133
Epoch 5/100
348/348 [==============================] - 0s 100us/step - loss: 0.3914 - acc: 0.8506 - val_loss: 0.4380 - val_acc: 0.8067
Epoch 6/100
348/348 [==============================] - 0s 97us/step - loss: 0.3877 - acc: 0.8391 - val_loss: 0.4396 - val_acc: 0.8000
Epoch 7/100
348/348 [==============================] - 0s 106us/step - loss: 0.3813 - acc: 0.8563 - val_loss: 0.4352 - val_acc: 0.8067
Epoch 8/100
348

In [9]:
## 모델 평가
res = model.predict(te)
def thres(a):
    if a < 0.5:
        return 0
    else:
        return 1
res = list(map(thres, res))
print(accuracy_score(test_y, res))

0.7850467289719626


In [10]:
model.evaluate(te, test_y)

214/214 [==============================] - 0s 47us/step


[0.47775495052337646, 0.7850467562675476]

In [11]:
## 모형을 쌓듯이 하는 방법의 같은 코드
model = models.Sequential()

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizers.Adam(lr=0.1),
             loss='binary_crossentropy',
             metrics=['acc'])

model.fit(tt, train_y.values,
         epochs=40,
         validation_split=0.3,
         callbacks=[cp, es])


Train on 348 samples, validate on 150 samples
Epoch 1/40
348/348 [==============================] - 0s 418us/step - loss: 0.6222 - acc: 0.7414 - val_loss: 0.5385 - val_acc: 0.8133
Epoch 2/40
348/348 [==============================] - 0s 103us/step - loss: 0.4812 - acc: 0.8103 - val_loss: 0.4748 - val_acc: 0.7933
Epoch 3/40
348/348 [==============================] - 0s 106us/step - loss: 0.4620 - acc: 0.8103 - val_loss: 0.4918 - val_acc: 0.7733
Epoch 4/40
348/348 [==============================] - 0s 106us/step - loss: 0.4373 - acc: 0.8132 - val_loss: 0.5039 - val_acc: 0.7933
Epoch 5/40
348/348 [==============================] - 0s 109us/step - loss: 0.4298 - acc: 0.8276 - val_loss: 0.5361 - val_acc: 0.7133
Epoch 6/40
348/348 [==============================] - 0s 112us/step - loss: 0.4295 - acc: 0.8218 - val_loss: 0.5303 - val_acc: 0.7467
Epoch 7/40
348/348 [==============================] - 0s 103us/step - loss: 0.3911 - acc: 0.8420 - val_loss: 0.5220 - val_acc: 0.7933
Epoch 8/40
348/3

In [12]:
## 보충자료

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=[32, 16], 
                   activation='relu',
                   alpha=0.01,
                   solver='adam',
                   early_stopping=True,
                   )

In [13]:
mlp.fit(train_X, train_y)

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=[32, 16], learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [14]:
accuracy_score(test_y, mlp.predict(test_X))

0.7149532710280374